In [37]:
import pandas as pd
import numpy as np
import os
import re
import spacy
import emoji
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from transformers import pipeline

In [4]:
path = ('C:/Users/BIGNETWORK/Desktop/PFE/dataset/resaux scraper')
files = os.listdir(path)
files

['alg_telcom_2_2k.xlsx',
 'facebook (1).xlsx',
 'facebook (2).xlsx',
 'instagram (1).xlsx',
 'instagram (2).xlsx',
 'instagram (3).xlsx',
 'instagram (4).xlsx',
 'instagram (5).xlsx',
 'instagram (6).xlsx',
 'instagram (7).xlsx',
 'instagram (8).xlsx',
 'instagram.xlsx']

In [109]:
files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls


['alg_telcom_2_2k.xlsx',
 'facebook (1).xlsx',
 'facebook (2).xlsx',
 'instagram (1).xlsx',
 'instagram (2).xlsx',
 'instagram (3).xlsx',
 'instagram (4).xlsx',
 'instagram (5).xlsx',
 'instagram (6).xlsx',
 'instagram (7).xlsx',
 'instagram (8).xlsx',
 'instagram.xlsx']

In [ ]:
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(f'C:/Users/BIGNETWORK/Desktop/PFE/dataset/resaux scraper/{f}')
    data.drop(data.index[0:0],inplace=True)
    df = df.append(data)

In [ ]:
print(df)

In [112]:
df.to_excel("resaux_comments.xlsx")

In [9]:
df0=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/resaux_comments.xlsx")
df1=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/hateonly_mehdi.xlsx")
df2=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/fb_comments.xlsx")
df3=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/hainer_ranim.xlsx")
df4=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/neutre_mehdi.xlsx")

In [10]:
df = pd.concat([df0, df1, df2, df3,df4], axis=0, ignore_index=True)

In [11]:
df

,comments,class
0,باينة كتبتوها نهار السبت حتى اليوم باش تبارتاج...,NaN
1,لوكان ستنيتو حتى نهار الخميس خير و درتو بيان,NaN
2,الزبون لا دخل له في البكالوريا,NaN
3,شركة.... تعجز عن وضع أجهزة تشويش بمراكز الإمتح...,NaN
4,دوماج عندي لافامي في الكونت هذا كون عبرتلكم عن...,NaN
...,...,...
10076,رجعولنا عرض 1500نتاع 40جيغا شهرين مشي شهر هكذا...,0.0
10077,انظروا إلى التعليقات يا دجيزي بفففففففففففف,0.0
10078,صح عيدك جيزي شوي اتهلى كونيكسيو ظعيفة وعروظ شو...,0.0
10079,عيدكم مبارك تقبل الله منا ومنكم صالح الأعمال يارب,0.0


In [12]:
df = df.sample(frac=1).reset_index(drop=True)
df

,comments,class
0,على الكونيكسيوا الزينة تاعكم الله يذلكم,-1.0
1,فعلولي عرض امتياز,NaN
2,Bon cnx ta3 djezzy mtsla7ch gae l pubg,NaN
3,حسام بن عبر,NaN
4,تم قطع الانترنت بشكل كامل،,NaN
...,...,...
10076,@sarra.hayat mabghach yetla3li,0.0
10077,Hahaha,0.0
10078,مناش عايشين في بلاد واحدة !,NaN
10079,Wallah ma tahchmou ya sraqin lmliha ki wlitou ...,NaN


In [13]:
df.to_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/dataset.xlsx")

In [14]:
df=df.dropna(subset = ['comments'])

In [15]:
df['comments'].isna().sum()

0

*** 
---
___

## <span style='font-family:Georgia'> Preprocessing of Text.

### <span style='font-family:serif'> Function to Remove Emojis

In [16]:
def strip_emoji(text):
    return emoji.replace_emoji(text,replace="")

### <span style='font-family:serif'> removing stop words

In [17]:
# Load stopwords for each language
with open('C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/Algerian-Arabic-stop-words.txt', 'r', encoding='utf-8') as f:
    stop_words_ar_dz = set([line.strip() for line in f])
stop_words_en = set(stopwords.words('english'))
stop_words_fr = set(stopwords.words('french'))
# Create a custom tokenizer for Arabic words
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
# Define a function to remove stop words from a text
def remove_stopwords(text):
    words = tokenizer.tokenize(text)
    words_filtered = []
    for word in words:
        if word not in stop_words_ar_dz and word not in stop_words_en and word not in stop_words_fr:
            words_filtered.append(word)
    return ' '.join(words_filtered)
tet = "Wlh loukan temdou alef jiga batel tqa3dou halazouuun."
tet = remove_stopwords(tet)
print(tet)

loukan temdou alef jiga batel tqa3dou halazouuun .


### <span style='font-family:serif'> Fucntion to remove special characters, URLs, duplicated letters, punctuations

In [18]:
# Define a function to perform the text cleaning operations
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation marks 
    text = re.sub(r'[@#&$]\w+', '', text)  # Remove special characters
    #text = re.sub(r'\d+', '', text)  # Remove numbers 
    text = re.sub(r'(\w)\1+', r'\1', text) # remove duplicated letters
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    return text

'''# Apply the clean_text function to the desired column(s) in the DataFrame
if df['comments'].dtype == 'object':
    df['comments_clean'] = df['comments'].apply(clean_text)'''


"# Apply the clean_text function to the desired column(s) in the DataFrame\nif df['comments'].dtype == 'object':\n    df['comments_clean'] = df['comments'].apply(clean_text)"

### <span style='font-family:serif'> Function to remove mutiple sequence spaces

In [19]:
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)

### <span style='font-family:serif'> Function to Preprocess the text by applying all above functions

In [20]:
def preprocess(text):
    text = strip_emoji(text)
    text = remove_stopwords(text)
    text = clean_text(text)
    text = remove_mult_spaces(text)
    return text


In [21]:
if df['comments'].dtype == 'object':
    df['comments_clean'] = df['comments'].apply(preprocess)

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_4440\2574735620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments_clean'] = df['comments'].apply(preprocess)


In [22]:
df.sample(25)

,comments,class,comments_clean
5402,نكتوها على رواحكم هذا واش نقولكم .,NaN,نكتوها رواحكم نقولكم
2675,البكالوريا بقاولها يومين وتخلص مازال الحال على...,NaN,البكالوريا بقاولها يومين وتخلص مازال الحال الا...
6454,PHANTOM ND ❤️🇩🇿✌️🔥💪,NaN,PHANTOM ND
4573,اذا ممكن مانقدرش نفليكسي من عندي لشخص آخر جربت...,0.0,اذا مكن مانقدرش نفليكسي عندي لشخص آخر جربت 70 ...
8419,تم نشر هذا المنشور يوم السبت بسبب ثقل الانترنت...,NaN,تم نشر المنشور يوم السبت بسب ثقل الانترنت زينة...
8930,Pubg mobile,NaN,Pubg mobile
9868,قال فيه فايد,NaN,فايد
7617,حتى ف الاعتذار متأخرين كالعادة,NaN,الاعتذار متأخرين كالعادة
47,عندي جيزي والحمد لله، ماشكيتلك ماشكيت لواحد اوخر,NaN,عندي جيزي والحمد له ماشكيتلك ماشكيت لواحد اوخر
3252,أنه النفاق يا هند,1.0,أنه النفاق هند


In [23]:
df['comments_clean'].isna().sum()

0

In [24]:
df['comments_clean'].replace('', np.nan, inplace=True)
missing_values = df[df['comments_clean'].isnull()]
#df=df.dropna(subset = ['comments_clean'])
len(missing_values)

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_4440\3493959318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments_clean'].replace('', np.nan, inplace=True)


380

In [25]:
df=df.dropna(subset = ['comments_clean'])

In [26]:
df= df.drop_duplicates(subset=["comments_clean"], keep="first")

In [28]:
from langdetect import detect
from difflib import SequenceMatcher

In [38]:
def lang_det(texte):
    langage = detect(texte)
    if langage not in ['en','fr']:
        langage = 'dz'
    
    return langage

In [69]:
# Define the mapping dictionary
mapping = {
    'a': 'ا', 'kh':'خ','sh':'ش','ch':'ش','b': 'ب', 'c': 'س', 'd': 'د', 'e': 'ي', 'f': 'ف',
    'g': 'ڨ', 'h': 'ه', 'i': 'ي', 'j': 'ج', 'k': 'ك', 'l': 'ل',
    'm': 'م', 'n': 'ن', 'o': 'و', 'p': 'ب', 'q': 'ق', 'r': 'ر',
    's': 'ص', 't': 'ت', 'u': 'و', 'v': 'ڥ', 'w': 'و', 'x': 'كس',
    'y': 'ي', 'z': 'ز','9':'ق','7':'ح'
}
# Define the translation function
def arabizi_to_arabic(text):
    # Replace each Arabizi letter with its corresponding Arabic alphabet dialect letter
    for letter, value in mapping.items():
        text = re.sub(letter, value, text)
    return text

In [71]:
# Example usage
arabizi_text = "Djezzy معنديش المال باش نفليكسي ابعتلي100دج"
arabic_text = arabizi_to_arabic(arabizi_text)
print(arabic_text)

Dجيززي معنديش المال باش نفليكسي ابعتلي100دج


In [30]:
# Load the language detection pipeline
lang_detect = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Text to classify
text = "Pourquoi vous limitez le débit ?."

# Run the language detection pipeline
result = lang_detect(text)

# Print the detected language
print(result[0]['label'])

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


fr


In [35]:
# Text to classify
text = "Vive mobilis"

# Run the language detection pipeline
result = lang_detect(text)

# Print the detected language
print(result[0]['label'])

pt


In [84]:
def similar(a, b):
# Utilisation de la méthode SequenceMatcher pour calculer la similarité entre deux chaînes de caractères
    return SequenceMatcher(None, a, b).ratio()
def detecter_langage(texte):
    # Détection de la langue avec langdetect
    langage = detect(texte)
    
   #  Si le texte est similaire à l'anglais, au français ou à l'arabe, il est classé comme dialecte algérien
    if langage in ['en', 'fr', 'ar']:
        similarites = {
            'en': similar(texte, 'english'),
            'fr': similar(texte, 'français'),
            'ar': similar(texte, 'العربية')
        }
        # Le texte est classé comme dialecte algérien s'il est plus similaire à une de ces langues qu'à l'autre
        if max(similarites.values()) == similarites[langage] or langage =='so':
            langage = 'dz'
    
    return langage

In [ ]:
txt = "Kanet 3endi beseh réseau fiha mekhsous bezaf f la willaya de Tizi ouzou apart le centre ville makanch réseau"
print(detecter_langage(txt))
  #  df['len'] = df['comments_nett'].apply(detecter_langage)

In [26]:
for index, row in df.iterrows():
    text = str(row['comments_clean'])
    if text:
        try:
            result = lang_detect(text)
            df.at[index, 'language'] = result[0]['label']
        except:
            pass

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_19160\2026574333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'language'] = result[0]['label']
Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors
